# Lecture 1: introduction to `qibo`

<center><img src="../figures/dall-e/merlion_qibo.png" alt="drawing" width="400"/></center>
<center><strong>Fig. 1:</strong> The merlion learning Qibo [DALL-E].</center>

### Introduction

During this course we are going to use `Qibo`, an open source framework for quantum computing. It provides us with an high level language which can be used to implement algorithms with both circuit-based and adiabatic computation approaches and, once the code is set up, it can be easily executed on various engines, including both classical and quantum hardware. 

<center><img src="../figures/qibo_ecosystem.svg" alt="drawing" width="800"/></center>

For more info about the whole framework one can have a look to the [`qibo` webpage](https://qibo.science/).

### Setup
We start installing `qibo` and then importing some useful primitives.

In [ ]:
# if you don't have already qibo and qibojit installed in your computer
! pip install qibo
! pip install qibojit

In [1]:
import qibo
from qibo import Circuit, gates, hamiltonians

A crucial step is the backend choice. In qibo four backends are provided, and can be used for different kind of applications.

<center><img src="../figures/backends.svg" alt="drawing" width="800"/></center>

The blue backends correspond to classical hardware, while the red one can be selected if we want to execute our algorithm directly on a quantum computer.

Since we want to do simulation here, we set the `qibojit` backend, which is the most suitable one if we want to run simulation with many qubits.

In [2]:
# with the following line we set the desired backend
qibo.set_backend(backend="qibojit")

[Qibo 0.2.4|INFO|2024-01-23 11:05:00]: Using qibojit (numba) backend on /CPU:0


In [5]:
# if qibojit is selected, we can specify the number of classical threads to be used
qibo.set_threads(16)

### Build my first `qibo` circuit

Now we are ready to code our first quantum circuit using `qibo`.

In [ ]:
# set the number of qubits
nqubits = 4

# we initialise the circuit using the Circuit class
c = Circuit(nqubits=nqubits)

print(c.draw())

Secondly, we add some gates to the circuit. A more detailed list of all the available gates can be found in the [`qibo` documentation](https://qibo.science/qibo/stable/api-reference/qibo.html#gates).

In [ ]:
# let's add some gates

# hadamards
c.add(gates.H(q=0))
c.add(gates.H(q=1))

# pauli's
c.add(gates.X(q=3))

# controlled gates
c.add(gates.CNOT(q0=0, q1=2))
c.add(gates.CZ(q0=1, q1=2))

# more freedom in controlling gates :)
c.add(gates.Y(q=3).controlled_by(0,1,2))

Once the circuit is completed, we can print some summary information.

In [ ]:
# summary info
print(c.summary())

In [ ]:
# circuit visualization
print(c.draw())

At this point we already can execute the circuit.
Note that, since we are playing with `nqubits = 4`, the final state will be a possible superposition of all the 4-bits combinations. We wrote a function in `scripts.utils` which can be used to create the list of all the possible bitstrings given the number of qubits:

In [ ]:
from scripts.utils import generate_bitstring_combinations

bitstrings = generate_bitstring_combinations(n=nqubits)
print(bitstrings)

Let's execute the circuit and verify which is the final state we get after the execution.

In [ ]:
# execute the circuit
outcome = c()

In [ ]:
# print the final state
print(outcome)

In this simple case we defined a circuit without adding measurement gates. What we get as result of such an execution is the exact state vector simulation of the final state.

Since we simulated the exact final state, we can also have access to the final amplitudes, and so to the probabilities of collect some of the states corresponding to the list of bitstrings.

In [ ]:
# print probabilities
probabilities = outcome.probabilities()
print(probabilities)

We can visualize them together with the bitstrings

In [ ]:
for bits, prob in zip(bitstrings, probabilities):
    print(f"Probability of state {bits}: {prob}")

#### Add measurements gates
We can, instead, add measurements gates in the end of the circuit, if we want to simulate a more realistic scenario affected by shot-noise.

With shot-noise we mean the natural variance of quantum system, due to the fact that we need to measure the final state after applying the gates. In a realistic scenario we need to repeat the circuit execution many times and use the collected results to calculate an estimation of the probabilities.

In [ ]:
# we initialise the same circuit using the Circuit class
c = Circuit(nqubits=nqubits)

# we add the same gates of before
c.add(gates.H(q=0))
c.add(gates.H(q=1))
c.add(gates.X(q=3))
c.add(gates.CNOT(q0=0, q1=2))
c.add(gates.CZ(q0=1, q1=2))
c.add(gates.Y(q=3).controlled_by(0,1,2))

# we also add measurements gates
c.add(gates.M(*range(nqubits)))

# print the new face of the circuit
print(c.draw())

Again, we can execute the circuit and collect the outcome.

At this time, we also specify a number of shots.

In [ ]:
outcome = c(nshots=1000)

We can still collect the exact probabilities

In [ ]:
# for all the qubits
print(f"All qubits:\n {outcome.probabilities()}\n")

# or for some specific qubit of the system
print(f"Some qubits:\n {outcome.probabilities(qubits=[0, 2])}")

We can also collect frequencies

In [ ]:
# frequencies
# set binary=True if you want the amplitude binary notation
print(f"Frequencies: {outcome.frequencies(binary=True)}\n")

#### Define a function to visualize the states

In [ ]:
import matplotlib.pyplot as plt

def visualize_states(counter):
    """Plot state's frequencies."""
        
    fig, ax = plt.subplots(figsize=(5, 5 * 6/8))
    
    ax.set_title('State visualization')
    ax.set_xlabel('States')
    ax.set_ylabel('#')
    
    for state in counter:
        ax.bar(state, counter[state], color='purple', alpha=0.5, edgecolor="black")

In [ ]:
# we collect the counter object containing the frequencies
frequencies = outcome.frequencies(binary=True)
visualize_states(frequencies)

#### Combining circuits into a single circuit 

It is possible to sum together two or more circuits (with the same number of qubits) into one.

In [ ]:
c1 = Circuit(3)
c1.add(gates.H(0))
c1.add(gates.CNOT(1,0))
c1.add(gates.Z(1))

c2 = Circuit(3)
c2.add(gates.H(2))
c2.add(gates.Z(1))
c2.add(gates.Y(1))
c2.add(gates.CZ(2,1))

print(f"Circuit n1:\n{c1.draw()}\n")
print(f"Circuit n2:\n{c2.draw()}\n")

# sum of c1 and c2
c3 = c1 + c2

print(f"Circuit n3:\n{c3.draw()}\n")

#### Let's simulate some entanglement

We can simulate the smallest entangling system in order to reproduce one of the Bell's states

$$ |b_1\rangle = \frac{|00\rangle + |11\rangle}{\sqrt{2}} \\. $$

To do this, we need to create a two-qubits circuit, lead one of the two qubits to a superposed state using an Hadamard gate and then apply a controlled-NOT gate to the second qubit using the superposed one as control.

In [ ]:
# two qubit circuit
c = Circuit(2)

c.add(gates.H(0))

c.add(gates.CNOT(q0=0, q1=1))

c.add(gates.M(*range(2)))

print(c.draw(), "\n")
print(c())

In [ ]:
outcome = c(nshots=1000)
frequencies = outcome.frequencies(binary=True)

# print the final state
print(f"Final state: {outcome}")

# visualize it
visualize_states(frequencies)

<div style="background-color: rgba(255, 105, 105, 0.3); border: 2.5px solid #000000; padding: 15px;">
    <strong>Exercise:</strong> implement the quantum circuits needed to prepare the other three Bell's states:
    $$ |b_2\rangle = \frac{|00\rangle - |11\rangle}{\sqrt{2}},\qquad |b_3\rangle = \frac{|10\rangle + |01\rangle}{\sqrt{2}},\qquad |b_4\rangle = \frac{|01\rangle - |10\rangle}{\sqrt{2}} \\. $$
</div>

### Parametrized gates

We can use parametric gates to manipulate the quantum state with some more freedom. 

The most commonly used parametric gates are rotations $R_k(\theta) = \exp [ -i \theta \sigma_k ] $, where $\sigma_k$ is one of the components of the Pauli's vector: $\vec{\sigma}=(I, \sigma_x, \sigma_y, \sigma_z)$.

In [ ]:
nqubits = 2
nlayers = 2

c = Circuit(nqubits=nqubits)

for l in range(nlayers):
    for q in range(nqubits):
        c.add(gates.RY(q=q, theta=0))
        c.add(gates.RZ(q=q, theta=0))
    c.add(gates.CNOT(q0=0, q1=nqubits-1))
c.add(gates.M(*range(nqubits)))

print(c.draw())

All the rotational angles are now set to zero, and the final state is equal to the initial state (which is $|0\rangle^{\otimes N}$ by default). We can play with the angles to modify the final state.

In [ ]:
# execute the circuit
outcome = c(nshots=1000)
frequencies = outcome.frequencies(binary=True)
# visualize the |0> state
visualize_states(frequencies)

In [ ]:
# visualize the current parameters (angles)
initial_parameters = c.get_parameters()
nparams = len(initial_parameters)

print(initial_parameters)

In [ ]:
# set a new random list of parameters
import numpy as np

# generate params according to N(0,1)
np.random.seed(42)
params = np.random.randn(nparams)

print(params)

c.set_parameters(params)

In [ ]:
c.get_parameters()

In [ ]:
frequencies = c(nshots=1000).frequencies()
visualize_states(frequencies)

### Hamiltonians

As final introductory step, let's see how to define and use an hamiltonian in Qibo.
We store a set of precomputed hamiltonians, which can be called and used to compute expectation values on the states we manipulate using circuits.

As a simple example, let's consider the following

In [ ]:
# define hamiltonian
h = hamiltonians.TFIM(nqubits=2)

In [ ]:
# visualize it
print(f"Matrix representation:\n {h.matrix}\n")

# some properties
print(f"Eigenvalues:\n {h.eigenvalues()}\n")
print(f"Eigenvectors:\n {h.eigenvectors()}\n")
print(f"Ground state:\n {h.ground_state()}\n")

We can calculate the expectation value of an hamiltonian over a target state.
We are going to use the final state $|\psi\rangle$ we obtain by executing the above circuit composted of rotations.

In [ ]:
# call the expectation < psi | H | psi >
state = c()

print(state)

In [ ]:
print(h.expectation(state.state()))